In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1. 문서 로딩
print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('./data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

# 2. 문서 분할
print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")

# 3. 벡터화 (HuggingFace 임베딩 사용)
print("==> 3. 벡터화 → 임베딩으로 변환")
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# 4. FAISS 벡터스토어 생성
print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 5. 검색기 설정
print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 성능 최적화
)

# 6. LLM 설정 (Ollama Qwen3)
print("===> 6. 생성 → Qwen3로 답변 생성")
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_predict=1500
)

# 7. 프롬프트 템플릿
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# 8. QA 체인 생성
print("===> 7. QA 체인 생성")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 9. 테스트 질문
questions = [
    "객체를 생성하는 방법은?",
    "코드패드는 무엇이고 어떻게 사용하나요?",
    "애플릿은 어떻게 실행하나요?"
]

# 10. 실행
print("===> 8. 질문 및 응답 실행")
for i, question in enumerate(questions, 1):
    print(f"\n【질문 {i}】 {question}")
    result = qa_chain.invoke({"query": question})
    print("📘 답변:", result["result"])
    for j, doc in enumerate(result["source_documents"][:2], 1):
        page = doc.metadata.get("page", "N/A")
        preview = doc.page_content[:70].replace('\n', ' ')
        print(f"🔎 참조 {j}: (p.{page}) {preview}...")


==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  76개 청크 생성 완료
==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
===> 5. 검색 → 질문과 유사한 문서 찾기
===> 6. 생성 → Qwen3로 답변 생성
===> 7. QA 체인 생성
===> 8. 질문 및 응답 실행

【질문 1】 객체를 생성하는 방법은?
📘 답변: <think>
Okay, let's tackle this question about creating objects in BlueJ. The user wants to know the method to create an object. From the provided document, I need to outline the steps based on the content.

First, the document mentions that when you click the OK button, the Staff object is created. The default name is staff_1, but the user can choose their own name. So the first step is to input the object's name, which is done via the "OK" button.

Then, the object appears on the Object Bench, which is shown in Figure 4. The user needs to select the constructor from the Class Popup menu to create the object. So the second step involves selecting the constructor.

The document also explains that the Person class is an abstract class, so you